In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D
from keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [17]:
#Reading the dataset from the csv file
data = pd.read_csv('./alphabets_28x28.csv')
data = data.dropna()
data = data.drop_duplicates()
print(data)

/tmp/ipykernel_63495/598006474.py:2: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./alphabets_28x28.csv')


       label row 1 col 1 pixel row 1 col 2 pixel row 1 col 3 pixel  \
0          A                 0                 0                 0   
1          A                 0                 0                 0   
2          A                 0                 0                 0   
3          A                 0                 0                 0   
4          A                 0                 0                 0   
...      ...               ...               ...               ...   
372406     Z                 0                 0                 0   
372416     Z                 0                 0                 0   
372427     Z                 0                 0                 0   
372438     Z                 0                 0                 0   
372449     Z                 0                 0                 0   

       row 1 col 4 pixel  row 1 col 5 pixel  row 1 col 6 pixel  \
0                      0                0.0                0.0   
1                      0   

In [18]:
#Given that four of the pixel columns are of a different type, converting them to numeric form
data["row 1 col 1 pixel"] = pd.to_numeric(data["row 1 col 1 pixel"])
data["row 1 col 2 pixel"] = pd.to_numeric(data["row 1 col 2 pixel"])
data["row 1 col 3 pixel"] = pd.to_numeric(data["row 1 col 3 pixel"])
data["row 1 col 4 pixel"] = pd.to_numeric(data["row 1 col 4 pixel"])
data.describe()

,row 1 col 1 pixel,row 1 col 2 pixel,row 1 col 3 pixel,row 1 col 4 pixel,row 1 col 5 pixel,row 1 col 6 pixel,row 1 col 7 pixel,row 1 col 8 pixel,row 1 col 9 pixel,row 1 col 10 pixel,...,row 28 col 19 pixel,row 28 col 20 pixel,row 28 col 21 pixel,row 28 col 22 pixel,row 28 col 23 pixel,row 28 col 24 pixel,row 28 col 25 pixel,row 28 col 26 pixel,row 28 col 27 pixel,row 28 col 28 pixel
count,217581.0,217581.0,217581.0,217581.0,217581.0,217581.0,217581.0,217581.0,217581.0,217581.0,...,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000,217581.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002767,0.002725,0.001912,0.001590,0.001172,0.001020,0.001057,0.001181,0.000409,0.000018
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.642121,0.676804,0.551249,0.548433,0.504455,0.418436,0.273368,0.438594,0.176433,0.008575
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,252.000000,226.000000,229.000000,228.000000,235.000000,194.000000,103.000000,198.000000,82.000000,4.000000


In [19]:
word_dict = {
    'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25
}

for i in data.index:
    data.loc[i,"label"] = word_dict[data["label"][i]]
print(data)

       label  row 1 col 1 pixel  row 1 col 2 pixel  row 1 col 3 pixel  \
0          0                  0                  0                  0   
1          0                  0                  0                  0   
2          0                  0                  0                  0   
3          0                  0                  0                  0   
4          0                  0                  0                  0   
...      ...                ...                ...                ...   
372406    25                  0                  0                  0   
372416    25                  0                  0                  0   
372427    25                  0                  0                  0   
372438    25                  0                  0                  0   
372449    25                  0                  0                  0   

        row 1 col 4 pixel  row 1 col 5 pixel  row 1 col 6 pixel  \
0                       0                0.0            

In [20]:
#Splitting the data into training and testing data and processing it to train the CNN
x = data.drop("label",axis=1).values
y = data["label"].values
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

x_train = x_train.reshape(-1,28,28,1).astype('float32')/255.0
print("New shape of train data:",x_train.shape)
x_test = x_test.reshape(-1,28,28,1).astype('float32')/255.0
print("New shape of test data:",x_test.shape)

New shape of train data: (174064, 28, 28, 1)
New shape of test data: (43517, 28, 28, 1)


In [21]:
categorical_train = to_categorical(y_train,num_classes=26).astype('int')
print("New shape of train labels:",categorical_train.shape)

categorical_test = to_categorical(y_test,num_classes=26).astype('int')
print("New shape of test labels:",categorical_test.shape)

New shape of train labels: (174064, 26)
New shape of test labels: (43517, 26)


In [22]:
#Setting up the neural network for training
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="valid"))
model.add(MaxPool2D(pool_size =(2,2),strides=2))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(26,activation="softmax"))

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(x_train,categorical_train,epochs=3,validation_data=(x_test,categorical_test))

/home/spidey/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
5440/5440 ━━━━━━━━━━━━━━━━━━━━ 66s 12ms/step - accuracy: 0.8814 - loss: 0.4125 - val_accuracy: 0.9793 - val_loss: 0.0749
Epoch 2/3
5440/5440 ━━━━━━━━━━━━━━━━━━━━ 69s 13ms/step - accuracy: 0.9799 - loss: 0.0704 - val_accuracy: 0.9819 - val_loss: 0.0633
Epoch 3/3
5440/5440 ━━━━━━━━━━━━━━━━━━━━ 68s 12ms/step - accuracy: 0.9852 - loss: 0.0511 - val_accuracy: 0.9804 - val_loss: 0.0689


In [23]:
#Reading the image and converting it to grayscale (ensuring format compatibility)
image = cv2.imread('./line_6.png')
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#commenting below line since test data does not require noise reduction
# image = cv2.GaussianBlur(image,(5,5),0)

In [24]:
#Function for predicting the character from given image using our CNN model
def predict_character(image):
    if(np.mean(image)==0):
        return [[-1]]
    image = image.astype("float32")/255.0
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    return prediction

In [25]:
#Function for padding the image to fit the given window size (in our case 28)
def pad_image(image,window_size):
    height,width = image.shape
    pad_height = (window_size-(height%window_size))%window_size
    pad_width = (window_size-(width%window_size))%window_size
    padded_image = cv2.copyMakeBorder(image,0,pad_height,0,pad_width,cv2.BORDER_CONSTANT,value=0)
    return padded_image

image = pad_image(image,28)

In [26]:
#Using sliding window algorithm to extract the character images
characters = []
height = image.shape[0]
width = image.shape[1]
for y in range(0,height,28):
    for x in range(0,width,28):
        window = image[y:y+28,x:x+28]
        characters.append(window)

In [27]:
#reversing the word_dict dictionary for getting char from prediction
reverse_word_dict = {v: k for k,v in word_dict.items()}
#Obtaining the corresponding extracted_phrase from the main image
extracted_phrase = ""
for char in characters:
    prediction = predict_character(char)
    if(prediction[0][0]==-1):
        predicted_character = ' '
    else:
        predicted_label = np.argmax(prediction)
        predicted_character = reverse_word_dict[predicted_label]
    extracted_phrase += predicted_character
    
print(extracted_phrase)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━

In [28]:
#Loading the dataset from the given csv file and segregate based on the columns
sentiment_df = pd.read_csv('./sentiment_analysis_dataset.csv')
lines = sentiment_df["line"].tolist()
sentiments = sentiment_df["sentiment"].tolist()

In [29]:
#Setting up the vectorizer to use in sentiment classification and training the Naive Bayes classifier
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lines)
sentiment_classifier = MultinomialNB()
sentiment_classifier.fit(X,sentiments)

MultinomialNB()

In [30]:
#Predicting the sentiment of the obtained phrase
def predict_sentiment(text):
    #using the vectorizer to process the input
    input = vectorizer.transform([text])
    predicted_sentiment = sentiment_classifier.predict(input)
    return predicted_sentiment[0]

predicted_sentiment = predict_sentiment(extracted_phrase)
print(f"The sentiment predicted by model is: {predicted_sentiment}")

The sentiment predicted by model is: Neutral
